In [5]:
import csv
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd 
import os
os.getcwd()

In [6]:
# function to check if the button is on the page, to avoid miss-click problem
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [7]:
def countpages(driver):
    
    html_soup = BeautifulSoup(driver.page_source, 'html.parser')

    #pull out list of page numbers
    if (check_exists_by_xpath('//a[contains(@class, "pageNum cx_brand_refresh_phase2")]')):
        post_content = html_soup.find_all('a', class_ = 'pageNum cx_brand_refresh_phase2')
    
        #find last element of text in list of page numbers
        last_div = None
        for last_div in post_content:pass
        if last_div:
            content = last_div.getText()
    else:
        content=1

    maxPageCount = int(content)
    return(maxPageCount)

In [8]:
def scrape(driver):
    pageCounter = 0
    while (pageCounter < maxPageCount):

        if (check_exists_by_xpath("//span[@class='location-review-review-list-parts-ExpandableReview__cta--2mR2g']")):
            # to expand the review 
            driver.find_element_by_xpath("//span[@class='location-review-review-list-parts-ExpandableReview__cta--2mR2g']").click()
            time.sleep(3)        
        
        container = driver.find_elements_by_xpath("//div[@class='location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw']")
        num_page_items = len(container)

                
        for j in range(num_page_items):
            # to save the rating
            string = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")
            data = string.split("_")
            # to save in a csv file. Each container line is a different element of the page that is saved- user, star rating, title, review, etc
            csvWriter.writerow([data[3], container[j].find_element_by_xpath(".//a[@class='location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT']").text.replace("\n", ""), container[j].find_element_by_xpath(".//div[@class='cPQsENeY']").text.replace("\n", ""), container[j].find_element_by_xpath(".//a[@class='ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC']").get_attribute("href")])
            #container[j].find_element_by_xpath(".//a[@class='styleguide-avatar-Avatar__avatar--2NStU ui_social_avatar inline']").get_attribute("href")])

        # to change the page
        if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')==True):
            # to click next
            driver.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
            time.sleep(5)
            csvFile.flush() #makes sure all scraped content is in csv file  
            pageCounter +=1
            print(pageCounter) 
        else:
            #to get reviews on last page
            csvFile.flush() #makes sure all scraped content is in csv file
            pageCounter +=1
            print("end")

    driver.close()

In [ ]:
#container[j].find_element_by_xpath(".//span[@class='location-review-review-list-parts-TripType__trip_type--3w17i']").text.replace("Trip type: ", "")])
                #container[j].find_element_by_xpath(".//span[@class='location-review-review-list-parts-EventDate__event_date--1epHa']").text.replace("Date of experience: ", ""),         


In [11]:
# Load in the dataframe
URLs = pd.read_csv("../data/processed/LA_combined_URLs.csv")
print(URLs)

                                                    URL
0     https://www.tripadvisor.com/Attraction_Review-...
1     https://www.tripadvisor.com/Attraction_Review-...
2     https://www.tripadvisor.com/Attraction_Review-...
3     https://www.tripadvisor.com/Attraction_Review-...
4     https://www.tripadvisor.com/Attraction_Review-...
...                                                 ...
1758  https://www.tripadvisor.com/Attraction_Review-...
1759  https://www.tripadvisor.com/Attraction_Review-...
1760  https://www.tripadvisor.com/Attraction_Review-...
1761  https://www.tripadvisor.com/Attraction_Review-...
1762  https://www.tripadvisor.com/Attraction_Review-...

[1763 rows x 1 columns]


In [12]:
URLs2 = URLs.drop_duplicates()
print(URLs2)

                                                    URL
0     https://www.tripadvisor.com/Attraction_Review-...
1     https://www.tripadvisor.com/Attraction_Review-...
2     https://www.tripadvisor.com/Attraction_Review-...
3     https://www.tripadvisor.com/Attraction_Review-...
4     https://www.tripadvisor.com/Attraction_Review-...
...                                                 ...
1758  https://www.tripadvisor.com/Attraction_Review-...
1759  https://www.tripadvisor.com/Attraction_Review-...
1760  https://www.tripadvisor.com/Attraction_Review-...
1761  https://www.tripadvisor.com/Attraction_Review-...
1762  https://www.tripadvisor.com/Attraction_Review-...

[1715 rows x 1 columns]


In [13]:
URLs2.drop(URLs2.head(749-34).index, inplace=True)#-2 accounts for change in numbering between "LA_combined" csv and URLs df. #32 bc duplicates?
#use to restart loop if hit error

/home/matthew/anaconda3/envs/insight/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
#skipped 279, 280, 282, 283. 278 partial (stop at ocean avenue after santa monica place)
#423- eaton canyon last one with user name not url
#may have skipped from 745-metro(770)
JUST NEED END OF HOLLYWOOD (749) (couldn't get the page to load and have 9760 so I'm stopping)

In [15]:
for url in URLs2['URL']:
    data = url.split("-")
    # open the file to save the review
    csvFile = open("/home/matthew/anaconda3/envs/insight/staycationLA/data/raw/reviews_LA_" + data[4] + "_url.csv", 'a')
    csvWriter = csv.writer(csvFile)
        # import the webdriver, chrome driver is recommended
    driver = webdriver.Chrome("/home/matthew/Downloads/chromedriver")
    # insert the tripadvisor's website of one attraction 
    driver.get(url)
    time.sleep(5)
    maxPageCount=countpages(driver)
    time.sleep(1)
    scrape(driver)

1


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=83.0.4103.97)


In [255]:
scrape(driver)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [9]:
def user_URL(driver):
    pageCounter = 0
    while (pageCounter < maxPageCount):   
        
        container = driver.find_elements_by_xpath("//div[@class='location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw']")
        num_page_items = len(container)

                
        for j in range(num_page_items):
            # to save the rating
            string = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")
            data = string.split("_")
            # to save in a csv file. Each container line is a different element of the page that is saved- user, star rating, title, review, etc
            csvWriter.writerow([data[3], container[j].find_element_by_xpath(".//a[@class='location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT']").text.replace("\n", ""), "", "", container[j].find_element_by_xpath(".//a[@class='ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC']").get_attribute("href")])
            #container[j].find_element_by_xpath(".//a[@class='styleguide-avatar-Avatar__avatar--2NStU ui_social_avatar inline']").get_attribute("href")])

        # to change the page
        if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')==True):
            # to click next
            driver.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
            time.sleep(5)
            csvFile.flush() #makes sure all scraped content is in csv file  
            pageCounter +=1
            print(pageCounter) 
        else:
            #to get reviews on last page
            csvFile.flush() #makes sure all scraped content is in csv file
            pageCounter +=1
            print("end")

    driver.close()

In [13]:
URLs3=URLs2.drop(URLs2.head(278-2).index, inplace=True)#-2 accounts for change in numbering between "LA_combined" csv and URLs df. #32 bc duplicates?
#use to restart loop if hit error
df = df.iloc[:1000]

/home/matthew/anaconda3/envs/insight/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


NameError: name 'df' is not defined

In [ ]:
open all reviews files, search for https://, add to list to make names
then merge rows if they have the same title and rating, change url to username

In [14]:
import glob
import os
os.chdir("/home/matthew/anaconda3/envs/insight/staycationLA/data/raw")
extension = 'csv'
all_filenames = [i for i in glob.glob('reviews_LA_*.{}'.format(extension))]
print(all_filenames)

['reviews_LA_Lancaster_Performing_Arts_Center.csv', 'reviews_LA_Valencia_Laser_Blast.csv', 'reviews_LA_East_West_Players_url.csv', 'reviews_LA_GoSedan_url.csv', 'reviews_LA_US_Open_of_Surfing_url.csv', 'reviews_LA_Amir_s_Garden_url.csv', 'reviews_LA_Ford_Theatres_url.csv', 'reviews_LA_Angeles_National_Forest_url.csv', 'reviews_LA_Leo_Carrillo_State_Park_and_Beach_url.csv', 'reviews_LA_Wades_Wines_url.csv', 'reviews_LA_AdventurePlex_url.csv', 'reviews_LA_Heritage_Park.csv', 'reviews_LA_RedCat_url.csv', 'reviews_LA_A_Moment_of_Peace_Massage_Therapy_url.csv', 'reviews_LA_Malibu_Presbyterian_Church_url.csv', 'reviews_LA_ArcLight_url.csv', 'reviews_LA_All_Saints_Episcopal_Church.csv', 'reviews_LA_O_Skin_Med_Spa_url.csv', 'reviews_LA_Little_Tokyo_url.csv', 'reviews_LA_Renaissance_Pleasure_Faire_url.csv', 'reviews_LA_International_Chess_Park.csv', 'reviews_LA_Malibu_Bluffs_Park_url.csv', 'reviews_LA_United_Artists_Theatre_url.csv', 'reviews_LA_Backbone_Trail_url.csv', 'reviews_LA_Westside_Pav

In [15]:
# Load in the dataframe
URLs = pd.read_csv("../../data/processed/LA_combined_URLs.csv")
print(URLs)

                                                    URL
0     https://www.tripadvisor.com/Attraction_Review-...
1     https://www.tripadvisor.com/Attraction_Review-...
2     https://www.tripadvisor.com/Attraction_Review-...
3     https://www.tripadvisor.com/Attraction_Review-...
4     https://www.tripadvisor.com/Attraction_Review-...
...                                                 ...
1758  https://www.tripadvisor.com/Attraction_Review-...
1759  https://www.tripadvisor.com/Attraction_Review-...
1760  https://www.tripadvisor.com/Attraction_Review-...
1761  https://www.tripadvisor.com/Attraction_Review-...
1762  https://www.tripadvisor.com/Attraction_Review-...

[1763 rows x 1 columns]


In [16]:
URLs1 = URLs.drop_duplicates()
print(URLs1)

                                                    URL
0     https://www.tripadvisor.com/Attraction_Review-...
1     https://www.tripadvisor.com/Attraction_Review-...
2     https://www.tripadvisor.com/Attraction_Review-...
3     https://www.tripadvisor.com/Attraction_Review-...
4     https://www.tripadvisor.com/Attraction_Review-...
...                                                 ...
1758  https://www.tripadvisor.com/Attraction_Review-...
1759  https://www.tripadvisor.com/Attraction_Review-...
1760  https://www.tripadvisor.com/Attraction_Review-...
1761  https://www.tripadvisor.com/Attraction_Review-...
1762  https://www.tripadvisor.com/Attraction_Review-...

[1715 rows x 1 columns]


In [17]:
url_list = URLs1['URL'].tolist()

In [209]:
df = pd.read_csv('reviews_LA_LA_Terp_Market_url.csv', index_col=None, names=['stars', 'title', 'text', 'user_old'])

In [210]:
df

,stars,title,text,user_old


In [212]:
df.empty

True

In [18]:
li2 = []

for filename in all_filenames:
    df = pd.read_csv(filename, index_col=None, names=['stars', 'title', 'text', 'user_old'])
    if (df. empty)==False: 
        if (len(df) == 0)==False:
            if ("https://" in df['user_old'])==False:        
                data=filename.split("_")
                data2="_".join(data[1:-2])
                URL = [match for match in url_list if data2 in match]
                li2.append(URL)
        
        #match list to list of urls?

In [19]:
li2

[[],
 [],
 [],
 ['https://www.tripadvisor.com/Attraction_Review-g33052-d1572640-Reviews-PAINT_LAB-Santa_Monica_California.html',
  'https://www.tripadvisor.com/Attraction_Review-g32422-d17708697-Reviews-LA_Coach_Inc-Gardena_California.html',
  'https://www.tripadvisor.com/Attraction_Review-g32422-d12515009-Reviews-LA_Terp_Market-Gardena_California.html',
  'https://www.tripadvisor.com/Attraction_Review-g32531-d3498530-Reviews-YouRaceLA-Irwindale_California.html',
  'https://www.tripadvisor.com/Attraction_Review-g32525-d13570849-Reviews-Shuttle_to_LAX-Inglewood_California.html',
  'https://www.tripadvisor.com/Attraction_Review-g32525-d13412546-Reviews-LAX_Firing_Range-Inglewood_California.html',
  'https://www.tripadvisor.com/Attraction_Review-g32655-d144145-Reviews-University_of_California_Los_Angeles_UCLA-Los_Angeles_California.html',
  'https://www.tripadvisor.com/Attraction_Review-g32655-d552939-Reviews-OUE_Skyspace_LA-Los_Angeles_California.html',
  'https://www.tripadvisor.com/Att

In [73]:
flat_list = [item for sublist in li2 for item in sublist]

In [74]:
len(flat_list)


15962

In [75]:
flat_list

['https://www.tripadvisor.com/Attraction_Review-g33052-d1572640-Reviews-PAINT_LAB-Santa_Monica_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32422-d17708697-Reviews-LA_Coach_Inc-Gardena_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32422-d12515009-Reviews-LA_Terp_Market-Gardena_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32531-d3498530-Reviews-YouRaceLA-Irwindale_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32525-d13570849-Reviews-Shuttle_to_LAX-Inglewood_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32525-d13412546-Reviews-LAX_Firing_Range-Inglewood_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d144145-Reviews-University_of_California_Los_Angeles_UCLA-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d552939-Reviews-OUE_Skyspace_LA-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d1

In [120]:
#remove duplicates
flat_list2 = list(dict.fromkeys(flat_list))

In [121]:
len(flat_list2)

1715

In [122]:
#flat_list2

In [123]:
[i for i, s in enumerate(flat_list2) if 'Hollywood_Bowl-' in s]

[767]

In [124]:
del flat_list2[:768]

In [125]:
flat_list2

['https://www.tripadvisor.com/Attraction_Review-g32655-d116886-Reviews-Travel_Town-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d104558-Reviews-Orpheum_Theater-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d12998011-Reviews-Lost_Spirits_Distillery-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d5584603-Reviews-Ahmanson_Theatre-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d181149-Reviews-Beverly_Center-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d156640-Reviews-Avila_Adobe-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d12200847-Reviews-Transformers_The_Ride_3D-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g32655-d156674-Reviews-Flower_Market-Los_Angeles_California.html',
 'https://www.tripadvisor.com/Attraction_Review-g326

In [126]:
for url in flat_list2:
    data = url.split("-")
    # open the file to save the review
    csvFile = open("/home/matthew/anaconda3/envs/insight/staycationLA/data/raw/reviews_LA_usernames" + data[4] + ".csv", 'a')
    csvWriter = csv.writer(csvFile)
        # import the webdriver, chrome driver is recommended
    driver = webdriver.Chrome("/home/matthew/Downloads/chromedriver")
    # insert the tripadvisor's website of one attraction 
    driver.get(url)
    time.sleep(5)
    maxPageCount=countpages(driver)
    time.sleep(1)
    user_URL(driver)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
end
1
2
3
4
5
6
7
8
9
10
end
1
2
3
4
5
6
7
end
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
end
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
end
1
2
3
4
5
6
7
8
9
10
11
12
13
end
1
2
3
4
end
1
2
3
4
5
end
1
2
3
4
5
6
7
8
9
10
11
12
13
end
1
2
3
4
5
6
7
end
1
2
3
4
5
6
end
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
end
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
end
1
2
3
4
5
6
7
8
9
end
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
end
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
end
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18

KeyboardInterrupt: 

In [119]:
user_URL(driver)

1
2
3
4
5
6
7
8
9
10
11
end
end
end
end
end
end
end
end


In [ ]:
import glob
import os
os.chdir("/home/matthew/anaconda3/envs/insight/staycationLA/data/raw")
extension = 'csv'
user_urls = [i for i in glob.glob('reviews_LA_usernames*.{}'.format(extension))]
print(user_urls)

In [ ]:
outer join with full dataframe

In [ ]:
li = []

for filename in all_filenames:
    df = pd.read_csv(filename, index_col=None, names=['stars', 'title', 'text', 'user', 'url'])
    if ("https://" in df['user'])==False:
        data=url.split("/")
        data2=data[-1]
        attraction_name='_'.join(data2)
        df['user_new'] = attraction_name        
        df.groupby(['stars', 'title'],as_index=False).agg(''.join)    
        #df.drop(['user_old'], axis=1)
        #li.append(df)

#complete = pd.concat(li, axis=0, ignore_index=True)
#export to csv
#complete.to_csv( "../processed/LA_combined_reviews.csv", index=False, encoding='utf-8-sig')

In [ ]:
    data = url.split("-")
    # open the file to save the review
    csvFile = open("/home/matthew/anaconda3/envs/insight/staycationLA/data/raw/reviews_LA_" + data[4] + "_url.csv", 'a')

In [277]:
# open the file to save the review
csvFile = open("/home/matthew/anaconda3/envs/insight/staycationLA/data/raw/LAreviews_gallery_JS.csv", 'a')
csvWriter = csv.writer(csvFile)

# import the webdriver, chrome driver is recommended
driver = webdriver.Chrome("/home/matthew/Downloads/chromedriver")
# insert the tripadvisor's website of one attraction 
driver.get("https://www.tripadvisor.com/Attraction_Review-g32954-d8836314-Reviews-Gallery_JS-Redondo_Beach_California.html")

#driver.get("https://www.tripadvisor.com/Attraction_Review-g32859-d127384-Reviews-The_Gamble_House-Pasadena_California.html")


In [23]:
String newWindow = driver.getWindowHandle();
 
driver.findElement(By.cssSelector(“body”)).sendKeys(Keys.CONTROL+”t”);
driver.get("Second URL");
 _255i5rcQ
    
    attraction_container = driver.find_elements_by_xpath("//div[@class='_2j03JUe9 MmIH_ltD _2JdZspdU']")
    num_attractions = len(attraction_container);
    
    for j in range(num_attractions):
        # to save the rating
        string = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")
        data = string.split("_")
        # to save in a csv file. Each container line is a different element of the page that is saved- user, star rating, title, review (would also like hometown, # reviews, # upvoted reviews, etc. but not needed so skipping for now)
        csvWriter.writerow([data[3], 
                           container[j].find_element_by_xpath(".//a[@class='location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT']").text.replace("\n", ""), 
                           container[j].find_element_by_xpath(".//div[@class='cPQsENeY']").text.replace("\n", ""),                                 
                           container[j].find_element_by_xpath(".//a[@class='ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC']").text.replace("\n", "")]) 
                            #container[j].find_element_by_xpath(".//span[@class='ui_icon map-pin-fill social-member-common-MemberHometown__icon--3WpeC']").text.replace("\n", "")])
                            #container[j].find_element_by_xpath(".//span[@class='default social-member-common-MemberHometown__hometown--3kM9S small']/span").text.replace("\n", "")])
                            #container[j].find_element_by_xpath(".//span[@class='default social-member-common-MemberHometown__hometown--3kM9S small']/descendant::span").text.replace("\n", "")])
                           #container[j].find_element_by_xpath(".//span[contains(@class, 'social-member-MemberHeaderStats__bold--3z3qh') and contains (text(), 'contributions')]").text.replace("\n", ""), 
                           #var admi = driver.FindElement(By.XPath("//span[contains(@class,'title') and contains(text(),'Administration')]"));
                           #container[j].find_element_by_xpath(".//span[contains(@class, 'social-member-MemberHeaderStats__bold--3z3qh') and contains (text(), 'helpful votes')]").text.replace("\n", "")
                           #])

SyntaxError: invalid syntax (<ipython-input-23-ac572c1afb8d>, line 1)